In [ ]:
import pandas as pd
import yfinance as yf

# -----------------------------
# Parameters
# -----------------------------
ticker = "VOO"
years = 10                 # longer history improves tail estimates
threshold = -0.015         # -1.5%

# -----------------------------
# Load daily data
# -----------------------------
hist = yf.Ticker(ticker).history(period=f"{years}y", auto_adjust=True)

# Ensure required columns
required_cols = {"Open", "Close"}
if not required_cols.issubset(hist.columns):
    raise ValueError(f"Missing columns. Found: {list(hist.columns)}")

# Keep weekdays only (defensive)
hist = hist[hist.index.dayofweek < 5]

# -----------------------------
# Build Monday open → Friday close weeks
# -----------------------------
hist["week"] = hist.index.to_period("W-FRI")

weekly = (
    hist
    .groupby("week")
    .agg(
        monday_open=("Open", lambda x: x.iloc[0]),
        friday_close=("Close", lambda x: x.iloc[-1])
    )
    .dropna()
)

# -----------------------------
# Compute weekly return
# -----------------------------
weekly["chg_%"] = weekly["friday_close"] / weekly["monday_open"] - 1

# -----------------------------
# Statistics
# -----------------------------
total_weeks = len(weekly)
count_breach = (weekly["chg_%"] <= threshold).sum()
prob_breach = count_breach / total_weeks

print(f"Total weeks analyzed: {total_weeks}")
print(f"Weeks with Friday close ≤ {threshold:.1%} below Monday open: "
      f"{count_breach} ({prob_breach:.2%})")

# -----------------------------
# Optional diagnostics
# -----------------------------
print("\nSummary of weekly returns:")
print(weekly["chg_%"].describe(percentiles=[0.01, 0.05, 0.10]))

print("\nMost recent weeks breaching -1.5%:")
print(
    weekly.loc[weekly["chg_%"] <= threshold]
    .tail(10)
    .assign(chg_pct=lambda x: (x["chg_%"] * 100).round(2))
)


Total weeks analyzed: 784
Weeks with Friday close ≤ -1.5% below Monday open: 123 (15.69%)

Summary of weekly returns:
count    784.000000
mean       0.002699
std        0.020440
min       -0.084927
1%        -0.054646
5%        -0.030345
10%       -0.023054
50%        0.003668
max        0.110814
Name: chg_%, dtype: float64

Most recent weeks breaching -1.5%:
                       monday_open  friday_close     chg_%  chg_pct
week                                                               
2025-02-15/2025-02-21   554.973604    545.045715 -0.017889    -1.79
2025-03-01/2025-03-07   541.538840    523.075867 -0.034094    -3.41
2025-03-22/2025-03-28   520.082723    506.342682 -0.026419    -2.64
2025-03-29/2025-04-04   500.900606    461.457794 -0.078744    -7.87
2025-04-12/2025-04-18   495.686471    479.677399 -0.032297    -3.23
2025-05-17/2025-05-23   535.912413    527.754211 -0.015223    -1.52
2025-07-26/2025-08-01   582.719684    568.212341 -0.024896    -2.49
2025-10-04/2025-10-10   61

C:\Users\Jonbf5\AppData\Local\Temp\ipykernel_28140\510406177.py:27: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["week"] = hist.index.to_period("W-FRI")
